In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import threading

import seaborn as sns
sns.set_style("darkgrid")

In [2]:
import tensorflow as tf
import scipy.io 
import matplotlib.pyplot as plt
import cv2
import keras
from glob import glob
import numpy as np
from tqdm import tqdm
import os
from PIL import Image
import pandas as pd
import cv2

from sklearn.model_selection import KFold
# from keras.preprocessing.image import ImageDataGenerator

# import keras_metrics

from keras.applications import mobilenet, resnet50 #, vgg16, inception_v3, resnet50, 
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, History

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

import logging
# logging.getLogger().setLevel(logging.DEBUG)
import pickle


import seaborn
seaborn.set_style("darkgrid")

import gc

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve

C:\ProgramData\Miniconda3\envs\keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from glob import glob

In [4]:
all_data_dir = 'E:\\Work/PathoBarIlan/Shlomi2018/'
# all_data_dir = '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018'

is_relative_path_csv = False
seed = 4221

pos_name_init = 'Cancer'
neg_name_init = 'Normal'

use_rgb = True # True=rgb, False=spectral
if use_rgb:
    file_ext = '.png'
else:
    file_ext = '.npy'
    
window_size = (200, 200)
shift = (100, 100)

In [20]:
w,h = window_size
if use_rgb:
    input_shape = (w,h,3)
else:
    input_shape = (w,h,40)
batch_size = 256

### loss

In [21]:
path = '/media/leetwito/Windows/Users/leetw/PycharmProjects/PathoBarIlan/my_models/k=2, lr5e-4/losses.txt'
with open(path, 'r') as f:
    lines = f.readlines()

FileNotFoundError: [Errno 2] No such file or directory: '/media/leetwito/Windows/Users/leetw/PycharmProjects/PathoBarIlan/my_models/k=2, lr5e-4/losses.txt'

In [22]:
train_loss = []
for line in lines:
#     line = str(line)
    train_loss.append(float(line.split(":")[2][:-1]))

NameError: name 'lines' is not defined

In [23]:
file_names = glob('/media/leetwito/Windows/Users/leetw/PycharmProjects/PathoBarIlan/my_models/k=2, lr5e-4/*.hdf5')
i=0
int(file_names[i].split("epoch")[1].split("-")[0])
file_names[0]

IndexError: list index out of range

In [24]:
val = []
epoch = []
for file in file_names:
    epoch_num = int(file.split("epoch")[1].split("-")[0])
    if epoch_num > 80:
        continue
    val.append(float(file.split("val_loss")[1].split("-")[0]))
    epoch.append(epoch_num)

In [25]:
df = pd.DataFrame({"val_loss":val, "epoch":epoch}).set_index("epoch").sort_index()
dff = pd.DataFrame([0] + train_loss, columns=["train"]).iloc[1:]
dfff = pd.concat([df, dff], axis=1)

In [26]:
dfff.rolling(5).mean().plot(figsize=(15, 8))

TypeError: Empty 'DataFrame': no numeric data to plot

### roc

In [27]:
# folders = glob('/media/leetwito/Windows/Users/leetw/PycharmProjects/PathoBarIlan/my_models/k*')
folders = glob('my_models/k*')

In [28]:
class XY():
    pass

In [41]:
def get_xy(n_batches, batch_size, sub, i, XY):
#             start = time()
    if use_rgb:

        X = [img_to_array(load_img(f.replace("npy", "png"), target_size=input_shape)) for f in sub.filename]
    else:
        X = [np.load(f) for f in sub.filename]
#             end = time()
#             print("batch load time: {}".format(end-start))

    X = batch_norm(np.stack(X))
    logging.debug("from file {}\nto file {}".format(sub.iloc[0].filename, sub.iloc[-1].filename))

    Y = sub.label.values
    Y = to_categorical(Y, num_classes=2)
    
    XY.X = X
    XY.Y = Y
    return None

In [42]:
def batch_norm(x):
#     print("x.shape:", x.shape)
    maxi = x.max(axis=1).max(axis=1)
#     print("maxi.shape:", maxi.shape)
    maxi = np.repeat(maxi[:, np.newaxis, : ], window_size[0], axis=1)
    maxi = np.repeat(maxi[:, np.newaxis, : ], window_size[1], axis=1)
    return x/maxi


def generator_from_df(df, batch_size, shuffle=True): 
    
    n_batches = df.shape[0]//batch_size
    while True:
        if shuffle:
            df_tmp = df.copy().sample(frac=1)  # frac=1 is same as shuffling df.
        else:
            df_tmp = df

        xy = XY()
        for i in range(n_batches):
            if i==0:
                sub = df_tmp.iloc[batch_size*i:batch_size*(i+1)]
                xy_tmp = XY()
                get_xy(n_batches, batch_size, sub, i, xy_tmp)
            j = i+1
            sub = df_tmp.iloc[batch_size*j:batch_size*(j+1)]
            t = threading.Thread(target=get_xy, args=(n_batches, batch_size, sub, j, xy))
            t.start()
            # Simple model, one input, one output.
            if i==0:
                yield xy_tmp.X, xy_tmp.Y
            else:
                yield xy.X, xy.Y
#                 print("after yield.")
                t.join()

        del df_tmp
        gc.collect()

In [43]:
def get_y_test_and_pred():
    y_tests = []
    y_preds = []
    for i in range(len(df_test)//batch_size):
        print(i, "out of", len(df_test)//batch_size)
        x, y = next(test_generator)
        print(mobilenet_model.evaluate(x, y))
        y_tests.append(y.argmax(axis=1))
        y_preds.append(mobilenet_model.predict(x)[:,1])
    
    y_test = np.stack(y_tests)
    y_pred = np.stack(y_preds)

    y_test = y_test.reshape((-1, 1))
    y_pred = y_pred.reshape((-1, 1))
    
    return y_test, y_pred

In [44]:
# tmp_val = (float(model.split("val_loss")[1].split("-")[0]))


In [45]:
folders = glob('my_models/k*')
all_ckpnts = glob(folders[0]+"/*.hdf5")
all_ckpnts[-1]

'my_models\\k_0\\model_rgb_weights_epoch99-val_loss0.003.hdf5'

In [46]:
# mobilenet_model = keras.models.load_model(all_ckpnts[0])

In [47]:
mobilenet_model = mobilenet.MobileNet(include_top=True, weights=None, input_shape=input_shape, classes=2, dropout=0.2)
optimizer = Adam(lr=5e-4) # 1e-3
mobilenet_model.compile(loss="binary_crossentropy", optimizer=optimizer) #  binary_crossentropy , categorical_crossentropy

mobilenet_model.load_weights(all_ckpnts[-1])

In [48]:
df_test = pd.read_pickle("df_test_%i"%0)
test_generator = generator_from_df(df_test, batch_size, shuffle=False)
x, y = next(test_generator)
x.shape

(256, 200, 200, 3)

In [49]:
mobilenet_model.predict(x).sum(axis=0)

array([ 55.334858, 200.66524 ], dtype=float32)

In [50]:
y

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.

In [51]:
%%time
mobilenet_model.predict(x).shape

Wall time: 2.01 s


(256, 2)

In [ ]:
mobilenet_model = mobilenet.MobileNet(include_top=True, weights=None, input_shape=input_shape, classes=2, dropout=0.2)
optimizer = Adam(lr=5e-4) # 1e-3
mobilenet_model.compile(loss="binary_crossentropy", optimizer=optimizer) #  binary_crossentropy , categorical_crossentropy
y_trues = []
y_preds = []


for i, folder in enumerate(folders):
    print("\n---------------------------------\n in %d file=%s\n---------------------------------"%(i, folder))
    df_test = pd.read_pickle("df_test_%i"%i)
    test_generator = generator_from_df(df_test, 16, shuffle=False)
    all_ckpnts = glob(folder+"/*.hdf5")
    val = 100
    for model in all_ckpnts:
        tmp_val = float(model.split("val_loss")[1].split(".")[0] + "." + model.split("val_loss")[1].split(".")[1])
        if tmp_val<val:
            file = model
            val = tmp_val
    print("read model")
    mobilenet_model.load_weights(file)
    y_test, y_pred = get_y_test_and_pred()
    y_trues.append(y_test)
    y_preds.append(y_pred)

In [ ]:
def plot_roc_curve(y_true, y_scores, figsize=(15, 15)):
    fig = plt.figure(figsize=figsize)
    fpr, tpr, threshold = roc_curve(y_true, y_scores)
    
    plt.plot(fpr, tpr)
    plt.plot([0, 1], [0, 1], "k--")
    
    plt.xlabel('TP rate', fontsize=20)
    plt.ylabel('FP rate', fontsize=20)
    plt.ylim([0.0, 1.1])
    plt.xlim([-0.02, 1.02])
    plt.title('RGB: ROC curve: AUC={0:0.3f}'.format(roc_auc_score(y_trues, y_preds)), fontsize=30)
    plt.rcParams.update({'font.size': 15})
        
    plt.axes()

    return fpr, tpr, threshold

def plot_precision_recall_curve(y_true, y_scores, figsize=(15, 8)):
    plt.figure(figsize=figsize)
    precision, recall, threshold = precision_recall_curve(y_true, y_scores)
    step_kwargs = ({'step': 'post'}
                   if 'step' in signature(plt.fill_between).parameters
                   else {})
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b')

    plt.xlabel('Recall', fontsize=20)
    plt.ylabel('Precision', fontsize=20)
    plt.ylim([0.0, 1.1])
    plt.xlim([0.0, 1.02])
    plt.title('RGB: Precision-Recall curve: AP={0:0.3f}'.format(
              average_precision), fontsize=30)
    plt.rcParams.update({'font.size': 15})


    return fpr, tpr, threshold

In [ ]:
y_trues = [i[:, 0] for i in y_trues] 
y_preds = [i[:, 0] for i in y_preds] 

In [ ]:
y_trues = np.concatenate(y_trues)
y_preds = np.concatenate(y_preds)

In [ ]:
pickle.dump([y_trues, y_preds], open('./my_models/all_y_trues-all_y_preds.pkl', 'wb'))

In [ ]:
fpr, tpr, threshold = plot_roc_curve(y_trues, y_preds)

In [ ]:
precision, recall, threshold = plot_precision_recall_curve(y_trues, y_preds)

In [ ]:
threshold[(precision+recall).argmax()]

In [ ]:
print(roc_auc_score(y_trues, y_preds))

In [ ]:
precision_score(y_trues>0.5, y_preds>0.5)

In [ ]:
recall_score(y_trues>0.5, y_preds>0.5)

In [ ]:
ts = 0.22
((y_trues>ts)==(y_preds>ts)).sum()/(y_trues.shape[0]), ((y_trues[y_trues==1]>ts)==(y_preds[y_trues==1]>ts)).sum()/(y_trues[y_trues==1].shape[0]), ((y_trues[y_trues==0]>ts)==(y_preds[y_trues==0]>ts)).sum()/(y_trues[y_trues==0].shape[0])

In [ ]:
y_trues[y_trues==1].shape

In [ ]:
average_precision_score(y_trues, y_preds)

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.utils.fixes import signature
from sklearn.metrics import precision_score, recall_score

In [ ]:
average_precision = average_precision_score(y_trues, y_preds)

In [ ]:
precision, recall, _ = precision_recall_curve(y_trues, y_preds)

In [ ]:
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.9, 1.05])
plt.xlim([0.9, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

In [ ]:
from sklearn.metrics import average_precision_score

In [ ]:
95 + 81 + 1815

In [ ]:
1177 + 222 -97 + 222 - 82